In [2]:
pip install symforce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Setup
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

from symforce.notebook_util import display
import symforce.symbolic as sf
from symforce import ops

In [4]:
# Identity definition
display(sf.Rot3())

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

In [5]:
# Symbolic definition
display(sf.Rot3.symbolic("R"))

<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>

In [6]:
# To/From rotation matrix

# Rotate about x-axis
theta = sf.Symbol("theta")
R_mat = sf.Matrix(
    [
        [1, 0, 0],
        [0, sf.cos(theta), -sf.sin(theta)],
        [0, sf.sin(theta), sf.cos(theta)],
    ]
)
R = sf.Rot3.from_rotation_matrix(R_mat)

display(R_mat)
display(R)  # Note the additional expressions required to avoid numerical errors
display(R.to_rotation_matrix())

⎡1    0        0   ⎤
⎢                  ⎥
⎢0  cos(θ)  -sin(θ)⎥
⎢                  ⎥
⎣0  sin(θ)  cos(θ) ⎦

<Rot3 <Q xyzw=[(1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)))*(sqrt(Max(0, 2 - 2*cos(theta)))/2 + Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))/2) + 2*sin(theta)*Min(1 - Max(0, sign(-2*cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)))), Max(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1))), 1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta))))))))))/(2*sqrt(Max(0, 2*cos(theta) + 2)) - 2*Min(1 - Max(0, sign(-2*cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), 1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1)), Min(1 - Max(0, sign(-cos(theta) + Max(1, 2*cos(theta) + 1, cos(theta)))), Max(1 - Max(0, 1 - Max(0, sig

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢ ⎛1   Min(1 - Max(0, 1 - Max(0, sign(Max(1, 2*cos(theta) + 1, cos(theta)) - 1
⎢ ⎜─ - ───────────────────────────────────────────────────────────────────────
⎢ ⎝2                                                                          
⎢                                                                             
⎢                                                   

In [7]:
# To/From Euler angles
R = sf.Rot3.from_yaw_pitch_roll(0, 0, theta)  # Yaw rotation only
ypr = R.to_yaw_pitch_roll()

display(R)
display(ops.StorageOps.simplify(list(ypr)))  # Simplify YPR expression

<Rot3 <Q xyzw=[sin(theta/2), 0, 0, cos(theta/2)]>>

In [8]:
# From axis-angle representation

# Rotate about x-axis
R = sf.Rot3.from_angle_axis(angle=theta, axis=sf.Vector3(1, 0, 0))

display(R)

<Rot3 <Q xyzw=[sin(theta/2), 0, 0, cos(theta/2)]>>

In [9]:
# Rotation defining orientation of body frame wrt world frame
world_R_body = sf.Rot3.symbolic("R")

# Point written in body frame
body_t_point = sf.Vector3.symbolic("p")

# Point written in world frame
world_t_point = world_R_body * body_t_point

display(world_t_point)

⎡   ⎛       2        2    ⎞                                                   
⎢p₀⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + p₁⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₂⋅(2⋅R_w⋅R_y + 2⋅R
⎢                                                                             
⎢                               ⎛      2        2    ⎞                        
⎢p₀⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + p₂⋅(-2⋅R_w⋅Rₓ + 2⋅R_y
⎢                                                                             
⎢                                                            ⎛      2        2
⎣p₀⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + p₁⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + p₂⋅⎝- 2⋅Rₓ  - 2⋅R_y 

      ⎤
ₓ⋅R_z)⎥
      ⎥
      ⎥
⋅R_z) ⎥
      ⎥
    ⎞ ⎥
 + 1⎠ ⎦

In [10]:
body_R_cam = sf.Rot3.symbolic("R_cam")
world_R_cam = world_R_body * body_R_cam

# Rotation inverse = negate vector part of quaternion
cam_R_body = body_R_cam.inverse()
display(body_R_cam)
display(cam_R_body)

<Rot3 <Q xyzw=[R_cam_x, R_cam_y, R_cam_z, R_cam_w]>>

<Rot3 <Q xyzw=[-R_cam_x, -R_cam_y, -R_cam_z, R_cam_w]>>

In [11]:
world_R_body_numeric = sf.Rot3.from_yaw_pitch_roll(0.1, -2.3, 0.7)
display(world_t_point.subs(world_R_body, world_R_body_numeric))

⎡-0.662947416398295⋅p₀ - 0.554353314451006⋅p₁ - 0.503182994394693⋅p₂ ⎤
⎢                                                                    ⎥
⎢-0.0665166116342196⋅p₀ + 0.713061539471145⋅p₁ - 0.697938952419008⋅p₂⎥
⎢                                                                    ⎥
⎣ 0.74570521217672⋅p₀ - 0.429226797490819⋅p₁ - 0.509596009450867⋅p₂  ⎦

### Poses
#### Poses are defined as a rotation plus a translation, and are constructed as such. We use the notation world_T_body to represent a pose that transforms from the body frame to the world frame.

In [12]:
# Symbolic construction
world_T_body = sf.Pose3.symbolic("T")
display(world_T_body)

<Pose3 R=<Rot3 <Q xyzw=[T.R_x, T.R_y, T.R_z, T.R_w]>>, t=(T.t0, T.t1, T.t2)>

In [13]:
# Construction from a rotation and translation

# Orientation of body frame wrt world frame
world_R_body = sf.Rot3.symbolic("R")

# Position of body frame wrt world frame written in the world frame
world_t_body = sf.Vector3.symbolic("t")

world_T_body = sf.Pose3(R=world_R_body, t=world_t_body)
display(world_T_body)

<Pose3 R=<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>, t=(t0, t1, t2)>

In [14]:
# Compose pose with a pose
body_T_cam = sf.Pose3.symbolic("T_cam")
world_T_cam = world_T_body * body_T_cam

# Compose pose with a point
body_t_point = sf.Vector3.symbolic("p")  # Position in body frame
# Equivalent to: world_R_body * body_t_point + world_t_body
world_t_point = world_T_body * body_t_point
display(world_t_point)

⎡   ⎛       2        2    ⎞                                                   
⎢p₀⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + p₁⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₂⋅(2⋅R_w⋅R_y + 2⋅R
⎢                                                                             
⎢                               ⎛      2        2    ⎞                        
⎢p₀⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + p₁⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + p₂⋅(-2⋅R_w⋅Rₓ + 2⋅R_y
⎢                                                                             
⎢                                                            ⎛      2        2
⎣p₀⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + p₁⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + p₂⋅⎝- 2⋅Rₓ  - 2⋅R_y 

           ⎤
ₓ⋅R_z) + t₀⎥
           ⎥
           ⎥
⋅R_z) + t₁ ⎥
           ⎥
    ⎞      ⎥
 + 1⎠ + t₂ ⎦

In [15]:
# Invert a pose
body_T_world = world_T_body.inverse()
display(world_T_body)
display(body_T_world)

<Pose3 R=<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>, t=(t0, t1, t2)>

<Pose3 R=<Rot3 <Q xyzw=[-R_x, -R_y, -R_z, R_w]>>, t=(-t0*(-2*R_y**2 - 2*R_z**2 + 1) - t1*(2*R_w*R_z + 2*R_x*R_y) - t2*(-2*R_w*R_y + 2*R_x*R_z), -t0*(-2*R_w*R_z + 2*R_x*R_y) - t1*(-2*R_x**2 - 2*R_z**2 + 1) - t2*(2*R_w*R_x + 2*R_y*R_z), -t0*(2*R_w*R_y + 2*R_x*R_z) - t1*(-2*R_w*R_x + 2*R_y*R_z) - t2*(-2*R_x**2 - 2*R_y**2 + 1))>

### Vectors and Matrices
#### Vectors and matrices are all represented using subclasses of sf.Matrix class, and can be constructed in several different ways as shown below.

In [16]:
# Matrix construction. The statements below all create the same 2x3 matrix object

# Construction from 2D list
m1 = sf.Matrix([[1, 2, 3], [4, 5, 6]])

# Construction using specified size + data
m2 = sf.Matrix(2, 3, [1, 2, 3, 4, 5, 6])

# sf.MatrixNM creates a matrix with shape NxM (defined by default for 6x6
# matrices and smaller)
m3 = sf.Matrix23(1, 2, 3, 4, 5, 6)
m4 = sf.Matrix23([1, 2, 3, 4, 5, 6])

# Construction using aliases
m5 = sf.M([[1, 2, 3], [4, 5, 6]])
m6 = sf.M(2, 3, [1, 2, 3, 4, 5, 6])
m7 = sf.M23(1, 2, 3, 4, 5, 6)
m8 = sf.M23([1, 2, 3, 4, 5, 6])

# Construction from block matrices of appropriate dimensions
m9 = sf.Matrix23.block_matrix([[sf.M13([1, 2, 3])], [sf.M13([3, 4, 5])]])

In [17]:
# Vector constructors. The statements below all create the same 3x1 vector object

# Construction from 2D list
v1 = sf.Matrix([[1], [2], [3]])

# Construction from 1D list. We assume a 1D list represents a column vector.
v2 = sf.Matrix([1, 2, 3])

# Construction using aliases (defined by default for 9x1 vectors and smaller)
v3 = sf.Matrix31(1, 2, 3)
v4 = sf.M31(1, 2, 3)
v5 = sf.Vector3(1, 2, 3)
v6 = sf.V3(1, 2, 3)

In [18]:
# Matrix of zeros
z1 = sf.Matrix23.zero()
z2 = sf.Matrix.zeros(2, 3)

# Matrix of ones
o1 = sf.Matrix23.one()
o2 = sf.Matrix.ones(2, 3)

In [19]:
zero_matrix = sf.Matrix33.zero()
identity_matrix = sf.Matrix33.eye()

# We could also write:
zero_matrix = ops.GroupOps.identity(sf.Matrix33)

display(zero_matrix)
display(identity_matrix)

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [20]:
# Matrix multiplication
m23 = sf.M23.symbolic("lhs")
m31 = sf.V3.symbolic("rhs")
display(m23 * m31)

⎡lhs₀ ₀⋅rhs₀ + lhs₀ ₁⋅rhs₁ + lhs₀ ₂⋅rhs₂⎤
⎢                                       ⎥
⎣lhs₁ ₀⋅rhs₀ + lhs₁ ₁⋅rhs₁ + lhs₁ ₂⋅rhs₂⎦

In [21]:
# Vector operations
norm = m31.norm()
squared_norm = m31.squared_norm()
unit_vec = m31.normalized()
display(unit_vec)

⎡           rhs₀           ⎤
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎢╲╱  rhs₀  + rhs₁  + rhs₂  ⎥
⎢                          ⎥
⎢           rhs₁           ⎥
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎢╲╱  rhs₀  + rhs₁  + rhs₂  ⎥
⎢                          ⎥
⎢           rhs₂           ⎥
⎢──────────────────────────⎥
⎢   _______________________⎥
⎢  ╱     2       2       2 ⎥
⎣╲╱  rhs₀  + rhs₁  + rhs₂  ⎦

In [22]:
m33 = 5 * sf.Matrix33.eye()  # Element-wise multiplication with scalar
display(m33.inv())  # Matrix inverse

⎡1/5   0    0 ⎤
⎢             ⎥
⎢ 0   1/5   0 ⎥
⎢             ⎥
⎣ 0    0   1/5⎦

### One of the most powerful operations can use matrices for is to compute jacobians with respect to other geo objects. By default compute jacobians with respect to the tangent space of the given object.

In [23]:
R0 = sf.Rot3.symbolic("R0")
R1 = sf.Rot3.symbolic("R1")
residual = sf.M(R0.local_coordinates(R1))
display(residual)

⎡2⋅(2⋅Min(0, sign(R0_w*R1_w + R0_x*R1_x + R0_y*R1_y + R0_z*R1_z)) + 1)⋅(R_0_w⋅
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                               ______________
⎢                                                              ╱              
⎢                                                            ╲╱  1 - Min(1.0, 
⎢                                                                             
⎢2⋅(2⋅Min(0, sign(R0_w*R1_w + R0_x*R1_x + R0_y*R1_y + R0_z*R1_z)) + 1)⋅(R_0_w⋅
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                               ______________
⎢                                                              ╱              
⎢                                                            ╲╱  1 - Min(1.0, 
⎢                                                                             
⎢2⋅(2⋅Min(0, sign(R0_w*R1_w + R0_x*R1_x + R0_y*R1_y 

In [24]:
jacobian = residual.jacobian(R1)
# The jacobian is quite a complex symbolic expression, so we don't display it for
# convenience.
# The shape is equal to (dimension of residual) x (dimension of tangent space)
display(jacobian.shape)

### General properties of geo objects
#### Storage operations
All geometric types implement the “Storage” interface. This means that they can:

1. Be serialized into a list of scalar expression(to_storage())

2. Be reconstructed from a list of scalar expressions (.from_storage())

3. Use common symbolic operations (symbolic construction, substitution, simplification, etc.)

In [25]:
# Serialization to scalar list
rot = sf.Rot3()
elements = rot.to_storage()
assert len(elements) == rot.storage_dim()
display(elements)

In [26]:
# Construction from scalar list
rot2 = sf.Rot3.from_storage(elements)
assert rot == rot2

In [27]:
# Symbolic operations
rot_sym = sf.Rot3.symbolic("rot_sym")
rot_num = rot_sym.subs(rot_sym, rot)

display(rot_sym)
display(rot_num)
display(rot_num.simplify())  # Simplify internal symbolic expressions
display(rot_num.evalf())  # Numerical evaluation

<Rot3 <Q xyzw=[rot_sym_x, rot_sym_y, rot_sym_z, rot_sym_w]>>

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1.00000000000000]>>

### Group operations
#### All geometric types also implement the “Group” interface, meaning that geometric objects:

1. Can be composed with objects of the same type to produce an object of the same type (.compose())

2. Have an identity element (.identity())

3. Can be inverted (.inverse())

4. Can be created to represent the relation between two other objects of the same type (.between())

In [28]:
# Construct two random rotations
R1 = sf.Rot3.random()
R2 = sf.Rot3.random()

# Composition
display(R1.compose(R2))  # For rotations this is the same as R1 * R2

<Rot3 <Q xyzw=[-0.571765785540102, 0.229547103730649, -0.249529515375072, 0.747079001586353]>>

In [29]:
# Identity
R_identity = sf.Rot3.identity()
display(R1)
display(R_identity * R1)

<Rot3 <Q xyzw=[0.370391570493066, -0.802750442423332, 0.439419511919046, 0.159098410556649]>>

<Rot3 <Q xyzw=[0.370391570493066, -0.802750442423332, 0.439419511919046, 0.159098410556649]>>

In [30]:
# Inverse
R1_inv = R1.inverse()
display(R_identity)
display(R1_inv * R1)

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

<Rot3 <Q xyzw=[0, 0, 0, 1.00000000000000]>>

In [31]:
# Between
R_delta = R1.between(R2)
display(R1 * R_delta)
display(R2)

<Rot3 <Q xyzw=[-0.467121244951332, 0.795059991729949, 0.00598208552672286, -0.386835322480046]>>

<Rot3 <Q xyzw=[-0.467121244951332, 0.795059991729949, 0.00598208552672283, -0.386835322480047]>>

### Lie Group operations
#### Rotations, poses, and matrices all implement the “LieGroup” interface, meaning that they each have a tangent space. There are many great references on Lie groups out there already, so instead of introducing them here, we recommend checking out Frank Dellaert’s, Ethan Eade’s, or JL Blanco’s tutorials. In SymForce, objects which are a Lie Group can:

1. Be used to compute the tangent space vector about the identity element (.to_tangent())

2. Be constructed from a tangent space vector about the identity element (.from_tangent())

3. Be perturbed by a tangent space vector about the given element (.retract())

4. Be used to compute the tangent space perturbation needed to obtain another given element (.local_coordinates())

5. Be used to compute a jacobian describing the relation between the underlying data of the object (e.g. a quaternion for a rotation) and the tangent space vector about the given element (.storage_D_tangent())

In [32]:
# To/From tangent space vector about identity element
R1 = sf.Rot3.random()
tangent_vec = R1.to_tangent()
R1_recovered = sf.Rot3.from_tangent(tangent_vec)

assert len(tangent_vec) == R1.tangent_dim()
display(R1)
display(R1_recovered)

<Rot3 <Q xyzw=[-0.535003333515050, 0.691446997308981, 0.00439867509309066, -0.485441175321586]>>

<Rot3 <Q xyzw=[0.535003333515050, -0.691446997308981, -0.00439867509309066, 0.485441175321586]>>

In [33]:
# Tangent space perturbations

# Perturb R1 by the given vector in the tangent space around R1
R2 = R1.retract([0.1, 2.3, -0.5])

# Compute the tangent vector pointing from R1 to R2, in the tangent space
# around R1
recovered_tangent_vec = R1.local_coordinates(R2)

display(recovered_tangent_vec)

In [34]:
# Jacobian of storage w.r.t tangent space perturbation

# We chain storage_D_tangent together with jacobians of larger symbolic
# expressions taken with respect to the symbolic elements of the object (e.g. a
# quaternion for rotations) to compute the jacobian wrt the tanget space about
# the element.
# I.e. residual_D_tangent = residual_D_storage * storage_D_tangent

jacobian = R1.storage_D_tangent()
assert jacobian.shape == (R1.storage_dim(), R1.tangent_dim())